# Génération de contenu musical au format MIDI avec miditoolkit

In [13]:
from miditoolkit import MidiFile, Instrument, Note
import random
import pretty_midi

# Partie 1

### Partie 1.1

In [4]:
midi_file = MidiFile()
print("Résolution par défaut :", midi_file.ticks_per_beat)

Résolution par défaut : 480


### Partie 1.2

In [6]:
midi_file = MidiFile()

instrument = Instrument(program=0, is_drum=False, name="Single Note Track")

note = Note(velocity=80, pitch=60, start=0, end=480)
instrument.notes.append(note)

midi_file.instruments.append(instrument)

midi_file.dump("single_note.mid")
print("Fichier MIDI avec une seule note créé : single_note.mid")

Fichier MIDI avec une seule note créé : single_note.mid


### Partie 1.3

In [7]:
def create_chromatic_scale(output_file, start_pitch=60, descending=False):
    midi_file = MidiFile()
    instrument = Instrument(program=0, name="Chromatic Scale")

    pitch_range = range(start_pitch, start_pitch + 12) if not descending else range(start_pitch + 11, start_pitch - 1, -1)
    time = 0

    for pitch in pitch_range:
        note = Note(velocity=80, pitch=pitch, start=time, end=time + 480)
        instrument.notes.append(note)
        time += 480

    midi_file.instruments.append(instrument)
    midi_file.dump(output_file)
    print(f"Gamme chromatique {'descendante' if descending else 'montante'} créée : {output_file}")

create_chromatic_scale("chromatic_scale_up.mid", start_pitch=60, descending=False)
create_chromatic_scale("chromatic_scale_down.mid", start_pitch=60, descending=True)


Gamme chromatique montante créée : chromatic_scale_up.mid
Gamme chromatique descendante créée : chromatic_scale_down.mid


### Partie 1.4

In [8]:
def create_scale(output_file, scale_intervals, start_pitch=60, descending=False):
    midi_file = MidiFile()
    instrument = Instrument(program=0, name="Custom Scale")

    intervals = scale_intervals[::-1] if descending else scale_intervals
    time = 0
    pitch = start_pitch

    for interval in intervals:
        note = Note(velocity=80, pitch=pitch, start=time, end=time + 480)
        instrument.notes.append(note)
        time += 480
        pitch += -interval if descending else interval

    midi_file.instruments.append(instrument)
    midi_file.dump(output_file)
    print(f"Gamme personnalisée {'descendante' if descending else 'montante'} créée : {output_file}")

create_scale("diatonic_scale_up.mid", scale_intervals=[2, 2, 1, 2, 2, 2, 1], descending=False)
create_scale("diatonic_scale_down.mid", scale_intervals=[2, 2, 1, 2, 2, 2, 1], descending=True)

Gamme personnalisée montante créée : diatonic_scale_up.mid
Gamme personnalisée descendante créée : diatonic_scale_down.mid


### Partie 1.5

In [10]:
def random_note_sequence(output_file, scale_intervals, start_pitch=60):
    midi_file = MidiFile()
    instrument = Instrument(program=0, name="Random Notes")

    pitches = [start_pitch]
    for interval in scale_intervals:
        pitches.append(pitches[-1] + interval)

    durations = [480, 960, 1920]
    time = 0

    for _ in range(10):
        pitch = random.choice(pitches)
        duration = random.choice(durations)
        note = Note(velocity=80, pitch=pitch, start=time, end=time + duration)
        instrument.notes.append(note)
        time += duration

    midi_file.instruments.append(instrument)
    midi_file.dump(output_file)
    print(f"Séquence aléatoire créée : {output_file}")

random_note_sequence("random_notes.mid", scale_intervals=[2, 2, 1, 2, 2, 2, 1])

Séquence aléatoire créée : random_notes.mid


### Partie 1.6

In [11]:
def random_sequence_with_silence(output_file, scale_intervals, start_pitch=60):
    midi_file = MidiFile()
    instrument = Instrument(program=0, name="Random Notes with Silence")

    pitches = [start_pitch]
    for interval in scale_intervals:
        pitches.append(pitches[-1] + interval)

    durations = [480, 960, 1920]
    time = 0

    for _ in range(10):
        is_silence = random.choice([True, False])
        duration = random.choice(durations)

        if not is_silence:
            pitch = random.choice(pitches)
            note = Note(velocity=80, pitch=pitch, start=time, end=time + duration)
            instrument.notes.append(note)

        time += duration

    midi_file.instruments.append(instrument)
    midi_file.dump(output_file)
    print(f"Séquence avec silences créée : {output_file}")

random_sequence_with_silence("random_notes_with_silence.mid", scale_intervals=[2, 2, 1, 2, 2, 2, 1])

Séquence avec silences créée : random_notes_with_silence.mid


### Ecoute du fichier Midi

In [14]:
midi_data = pretty_midi.PrettyMIDI("random_notes_with_silence.mid")
for instrument in midi_data.instruments:
    print(f"Instruments : {instrument.name}")

Instruments : Random Notes with Silence
